sourced from [ReactAgent.ipynb](https://colab.research.google.com/drive/1XYNaGvEdyKVbs4g_Maffyq08DUArcW8H?usp=sharing#scrollTo=xEY6G0JtXuMo)

In [ ]:
import os, openai
from dotenv import load_dotenv, find_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
assert openai.api_key, 'Please set OPENAI_API_KEY!'

github_token = os.getenv('GITHUB_TOKEN')
assert github_token

Define some simple tools to test agent

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool], llm=llm, verbose=True)

response = agent.chat("What is 20+(2*4)? Calculate step by step.")

print(response)

In [ ]:
while True:
    user_input = input()
    if not user_input:
        break
    print(f"User: {user_input}")
    response = agent.chat(user_input)
    print(f"Agent: {response}")

In [ ]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Create the vector index store for Uber and Lyft

In [ ]:
from llama_index.core import SimpleDirectoryReader

lyft_docs = SimpleDirectoryReader(
    input_files=["../data/lyft_2021.pdf"]
).load_data()

uber_docs = SimpleDirectoryReader(
    input_files=["../data/uber_2021.pdf"]
).load_data()

brainwave_docs = SimpleDirectoryReader(
    input_files=["../data/ISCA18-Brainwave-CameraReady.pdf"]
).load_data()

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

# build index
lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
uber_index = VectorStoreIndex.from_documents(uber_docs,  show_progress=True)
brainwave_index = VectorStoreIndex.from_documents(brainwave_docs, show_progress=True)

# persist index
lyft_index.storage_context.persist(persist_dir="./storage/lyft")
uber_index.storage_context.persist(persist_dir="./storage/uber")
brainwave_index.storage_context.persist(persist_dir="./storage/brainwave")

Create the query engines w/ top_k set to 3

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)
brainwave_engine = brainwave_index.as_query_engine(similarity_top_k=3)

Define the query engine tools, each tied to the query engine / vector index with descriptions of each 

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),

    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),

    QueryEngineTool(
        query_engine=brainwave_engine,
        metadata=ToolMetadata(
            name="ISCA18",
            description=(
                "Provides information about the Brainwave NPU hardware accelerator. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

Create the LLM and agent to use

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

Query the agent

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

In [ ]:
response = agent.chat(
    "What is so special about Brainwave?"
)

In [ ]:
while True:
    user_input = input()
    if not user_input:
        break
    print(f"User: {user_input}")
    response = agent.chat(user_input)
    print(f"Agent: {response}")

## Github Repo Reader

In [ ]:
!pip install llama-index-readers-github --upgrade
!pip install nest_asyncio httpx

In [ ]:
# This is due to the fact that we use asyncio.loop_until_complete in
# the DiscordReader. Since the Jupyter kernel itself runs on
# an event loop, we need to add some help with nesting
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.github import GithubRepositoryReader
from IPython.display import Markdown, display
import os

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.readers.github import GithubRepositoryReader, GithubClient

github_client = GithubClient(github_token=os.environ["GITHUB_TOKEN"], verbose=True)

reader1 = GithubRepositoryReader(
    github_client=github_client,
    owner="onnx",
    repo="turnkeyml",
    use_parser=False,
    verbose=True,
    filter_directories=(
        ["docs"],
        GithubRepositoryReader.FilterType.INCLUDE,
    ),
    filter_file_extensions=(
        [
            ".png",
            ".jpg",
            ".jpeg",
            ".gif",
            ".svg",
            ".ico",
            "json",
            ".ipynb",
        ],
        GithubRepositoryReader.FilterType.EXCLUDE,
    ),
)

repo_docs = reader1.load_data(branch="main")

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

# build index
repo_index = VectorStoreIndex.from_documents(repo_docs, show_progress=True)

# persist index
repo_index.storage_context.persist(persist_dir="./storage/repo")
repo_engine = repo_index.as_query_engine(similarity_top_k=3)

In [ ]:
print(repo_engine.query("how do i install turnkeyml repo?"))

In [ ]:
print(repo_engine.query("how do i install dependencies?"))

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=repo_engine,
        metadata=ToolMetadata(
            name="repo",
            description=(
                "Provides information about code repository. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

llm = OpenAI(model="gpt-4")
openai_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [ ]:
hint = "(do not answer implictly, instead use the readme tool)"
queries = [
    "give me the link to Turnkey tests",
    "give me the link to Turnkey instructions",
    "what is TurnkeyML's mission?",
    "how do I get started?",
    "where is the installation guide?",
    "how about tutorials?",
    "what use-cases are there?",
    "execute the demo.",
    "what was it benchmarked on?",
    "generated a report, collect the statistics and summarize in a spreadsheet",
]
queries = [
    "how do i install dependencies?",
    "generte the commands to install dependencies"
]
for query in queries:
    print(f"Query: {query}")
    response = openai_agent.chat(f"{query}\n{hint}")
    print('-------------------------------------------------------------------')